In [ ]:
import torch
from torch import nn
from torch.utils.data import random_split, DataLoader
from torchvision.transforms import Compose, ToTensor, Resize
from torchvision.datasets import ImageFolder
from torchsummary import summary
import math
import time
import os
import matplotlib.pylab as plt
from torch.optim import Adam
from torchvision.models import resnet50

In [ ]:
# set the device we will be using to train the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# !wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip
# !unzip nature_12K.zip

In [ ]:
training_data_path = "inaturalist_12K/train/"
testing_data_path = "inaturalist_12K/val/"

In [8]:
model = resnet50(weights=None)

In [9]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
def train(learning_rate, epochs, batch_size, train_set, validation_set):

  def average(arr):
    if len(arr)==0:
      return 0
    return math.fsum(arr) / len(arr)

  training_loss = training_accuracy = validation_loss = validation_accuracy = []

  train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
  val_dataloader = DataLoader(validation_set, batch_size=batch_size, shuffle=False)

  model = resnet50(pretrained=True)
  num_classes = 10
  num_ftrs = model.fc.in_features
  model.fc = nn.Linear(num_ftrs, num_classes)

  optimizer = Adam(model.parameters(), lr = learning_rate)
  criterion = nn.NLLLoss()

  for epoch in range(0, epochs):

    train_loss = val_loss = train_accuracy = val_accuracy = []

    model.train()
    for (x, y) in train_dataloader:
      (x, y) = (x.to(device), y.to(device))
      pred = model(x)

      loss = criterion(pred, y)
      accuracy = (pred.argmax(1) == y).type(torch.float).sum().item()
      train_loss.append(loss.item())
      train_accuracy.append(accuracy)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    with torch.no_grad():
      model.eval()
      for (x, y) in val_dataloader:
        (x, y) = (x.to(device), y.to(device))
        pred = model(x)

        loss = criterion(pred, y)
        accuracy = (pred.argmax(1) == y).type(torch.float).sum().item()
        val_loss.append(loss.item())
        val_accuracy.append(accuracy)

    training_loss.append(average(train_loss))
    training_accuracy.append(average(train_accuracy))
    validation_loss.append(average(val_loss))
    validation_accuracy.append(average(val_accuracy))
    print("Epoch: {}/{}    Train loss: {:.6f}, Train accuracy: {:.4f}    Val loss: {:.6f}, Val accuracy: {:.4f}".format(epoch + 1, epochs, training_loss[-1], training_accuracy[-1], validation_loss[-1], validation_accuracy[-1]))
  return model, training_loss, training_accuracy, validation_loss, validation_accuracy